In [2]:
!pip install simpletransformers
!pip install torch
!pip install sentence_transformers

You should consider upgrading via the '/Users/gokhanhas/miniforge3/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/gokhanhas/miniforge3/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/gokhanhas/miniforge3/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
# Gokhan Has - Experiment 03 - A
import os
import ast
import pandas as pd
import numpy as np
import statistics
import scipy

from simpletransformers.ner import NERModel
from operator import itemgetter
from scipy import spatial
from itertools import chain
from sentence_transformers import SentenceTransformer

In [4]:
def read_csv(filepath):
    if os.path.splitext(filepath)[1] != '.csv':
        return  # or whatever
    seps = [';', '\t']  # ',' is default
    encodings = [None, 'utf-8', 'ISO-8859-1', 'utf-16', 'ascii']  # None is default
    for sep in seps:
        for encoding in encodings:
            try:
                return pd.read_csv(filepath, encoding=encoding, sep=sep)
            except Exception:  # should really be more specific
                pass
    raise ValueError("{!r} is has no encoding in {} or seperator in {}"
                     .format(filepath, encodings, seps))

In [5]:
# Install NER model
model_NER = NERModel('bert', '../ner_model/', use_cuda=False)

In [6]:
# Read the dataset and initialize dataset values ...
df = read_csv('../datasets/exp2_dataset.csv')

all_sentences = []
oldVal = 0
newVal = 1
for index in range(0, len(df)):
    newVal = df['SENTENCE_ID'][index]
    if not oldVal == newVal:
        all_sentences.append( {'index': df['SENTENCE_ID'][index], 'sentence': df['SENTENCE'][index]})
        oldVal = newVal

In [7]:
# Get the query sentence
query_sentence = "insan öldürmenin cezası nedir"

# Get the query sentence list
query_sentence_list = query_sentence.split(' ')

# Get the query sentence named entities
prediction, model_output = model_NER.predict([query_sentence])
prediction = prediction[0]

print(prediction)

o_element_list = []
ner_query_list = []
for index, tup in enumerate(prediction):
    if not tup[query_sentence_list[index]] == 'O':
        ner_query_list.append(tup)
    else:
        o_element_list.append(query_sentence_list[index])
        
prediction = ner_query_list


[{'insan': 'B-ELEMENT_OF_CRIME'}, {'öldürmenin': 'I-ELEMENT_OF_CRIME'}, {'cezası': 'I-PUNISHMENT'}, {'nedir': 'O'}]


In [8]:
# Get the query sentence's word's embedding by using ../ner_model. 
query_word_embeddings_list = []

words = []
new_sentences = []
print(query_sentence_list)
for query_index, query_word in enumerate(query_sentence_list):
    if not query_word in o_element_list:
        print(query_word)
        query_word_embeddings = model_output[0][query_index][query_word][0]
        query_word_embeddings_list.append(query_word_embeddings)

['insan', 'öldürmenin', 'cezası', 'nedir']
insan
öldürmenin
cezası


In [9]:
# In this cell, compare dataset's named entity's embeddings and query sentence's word's embeddings...

for i in range(0, len(o_element_list)):
    query_sentence_list.remove(o_element_list[i])


results_list = []

oldIndex = 1
newIndex = 1
for k in range(0, len(prediction)):
    control_tag = prediction[k][query_sentence_list[k]]
    result_list = []
    cumle_list = []
    dataset_tag = ""
    for index in range(0, len(df)):
        newIndex = df['SENTENCE_ID'][index] 
        if not (newIndex == oldIndex):
            if len(cumle_list) == 0:
                result_list.append({'sentence_id' : oldIndex, 'distance': 0.0})
            else:
                avg = statistics.mean(cumle_list)
                result_list.append({'sentence_id' : oldIndex, 'distance': avg})
            # result_list.append(cumle_list)
            cumle_list = []
            oldIndex = newIndex
            
        dataset_tag = (df['TAG'][index])[2:len(df['TAG'][index])]
        # print(newIndex, df['WORD'][index], dataset_tag)
        if dataset_tag == control_tag[2:len(control_tag)]:
            # convert list string to list
            veriset_emb = ast.literal_eval(df['EMBEDDING'][index])
            # find cosine distance between query word and dataset word tag
            distances = 1 - spatial.distance.cosine(veriset_emb, query_word_embeddings_list[k])
            #cumle_list.append({dataset_tag : distances, 'sentence_id': df['SENTENCE_ID'][index]})
            cumle_list.append(distances)
   
    oldIndex = 1  
    results_list.append(result_list)
        

end_results_list = []

for i in range(0, len(results_list)):
    newlist = sorted(results_list[i], key=itemgetter('sentence_id'))
    end_results_list.append(newlist)


if not len(end_results_list) == 0:
    similarity_value_list = [0] * len(end_results_list[0])
    size = len(prediction)

    for i, list_dict in enumerate(end_results_list):
        for index, one_dict in enumerate(list_dict):   
            dict_values = list(one_dict.values())
            similarity_value_list[index] = similarity_value_list[index] + dict_values[1]

    similarity_value_list = [number / size for number in similarity_value_list]
    #print(similarity_value_list)


    results_dict = []
    for i, list_dict in enumerate(end_results_list):
        for index, one_dict in enumerate(list_dict):
            dict_values = list(one_dict.values())
            results_dict.append({'sentence_id': dict_values[0], 'distance': similarity_value_list[index]})
        break

    #print(results_dict)


    results_dict_2 = sorted(results_dict, key=lambda x: float(x['distance']), reverse=True)    
    result_100 = results_dict_2[:100]
    
    print(result_100)

    son = []
    for i, _dict in enumerate(result_100):
        for index, val in enumerate(all_sentences):
            if val['index'] == _dict['sentence_id']:
                son.append({'sentence' : val['sentence'], 'distance': _dict['distance']})

    for i, val in enumerate(son):
        print(i+1, val)
        print("\n")
else:
    print('!!! NO ENTITY !!! CANNOT COMPARE !!!')

[{'sentence_id': 595, 'distance': 0.8542695787211273}, {'sentence_id': 394, 'distance': 0.8485863087272166}, {'sentence_id': 398, 'distance': 0.8485863087272166}, {'sentence_id': 387, 'distance': 0.8480413512669612}, {'sentence_id': 1041, 'distance': 0.8426662962029869}, {'sentence_id': 359, 'distance': 0.8411173594048303}, {'sentence_id': 1048, 'distance': 0.8244669191556248}, {'sentence_id': 1012, 'distance': 0.8242795155225132}, {'sentence_id': 741, 'distance': 0.8239870781911091}, {'sentence_id': 1060, 'distance': 0.8210624410308242}, {'sentence_id': 417, 'distance': 0.820106871137793}, {'sentence_id': 329, 'distance': 0.8184711266274508}, {'sentence_id': 331, 'distance': 0.8184711266274508}, {'sentence_id': 1028, 'distance': 0.8181648756731678}, {'sentence_id': 411, 'distance': 0.8151231633808299}, {'sentence_id': 685, 'distance': 0.815038845712189}, {'sentence_id': 1081, 'distance': 0.8132996432209324}, {'sentence_id': 1030, 'distance': 0.8118824573115534}, {'sentence_id': 1049, 

In [10]:
### SBERT ###

# Install BERT model by using SentenceTransformer package.
model = SentenceTransformer('dbmdz/bert-base-turkish-128k-cased')

In [11]:
sentences = []
for i, val in enumerate(son):
    sentences.append(val['sentence'])


In [ ]:
sentence_embeddings = model.encode(sentences)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

In [ ]:
query = query_sentence

queries = [query]
query_embeddings = model.encode(queries)

number_top_matches = 10 

print("Deney3 Birinci Kısım Arama Sonuçları")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("ARANAN CÜMLE:", query)
    print("\nEN YÜKSEK 10 SONUÇ GÖSTERİLİYOR :\n\n")
    
    for idx, distance in results[0:number_top_matches]:
        print(sentences[idx].strip(), "(Cosine Score: %.4f)\n" % (1-distance))
        